In [ ]:
# This script downloads df_emails dat frame which cotains one row per email, with cleaned email body and infered owner. In/out
# info about the emails follows based on whther folder name matches the name in X-from (dist1=0 or dist2=0)

# Then script processes all emails bodies to bags of words, thereby removing standard English words, digits, spaces and special
# characters, words with 1 or 2 letters, bring nouns to singular form, removes sufix where appl. with appropriate nested list 
# as result. 

# Subsequently we determine tfidf values per word/email combination and remove all words with a tfidf bellow a threshold level
# from each single document. Word is removed from dictionary al together only if it haf a low tfidf in IN ALL DOCUMENTS!

# We chose the tfidf threshold such that we keep not to many words. Chpice to keep about 15.000 words is a bit arbitrary
# at this point. A way to determine óptimal'number of words is to measure perplexity of our topic model, against
# choices of tfidf threshold together with number of topics. Due to run time of lda model we will keep the tfidf threshold
# choice at this one level only but we will try different options for number fo topics in our next script.

In [ ]:
import pandas as pd
df_emails=pd.read_pickle('/notebooks/LDA models and data/Data Frames and lists/df_emails.plk')
df_emails0=pd.read_pickle('/notebooks/LDA models and data/Data Frames and lists/df_emails0.plk')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns; sns.set_style('whitegrid')
# import wordcloud

# Network analysis
# import networkx as nx


# from subprocess import check_output

# NLP
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer

# LDA
import gensim
from gensim import corpora

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.lda import LDA
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Function which removes punctuation and special characters, digits as weel as English words which are too common.
# In addition 
def clean_text(text):
    stop = set(stopwords.words('english'))
    stop.update(("to","cc","subject","http","from","sent","aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()) )])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    stem = " ".join(porter.stem(token) for token in normalized.split())
    two_letter_words_free = " ".join([i for i in stem.split() if len(i)>=3])
    
#     return normalized
    return two_letter_words_free

In [ ]:
clean_text(' !This is a 45.98*demonstrating. Of af actions ji of this function\'s $#wORKing/')

In [ ]:
# create a list of processed email texts needed to feed into gensim functions
text_clean=[]
for text in df_emails['body']:
    text_clean.append(clean_text(text).split())
    
# create a list of processed email texts needed to feed into gensim functions
# df_emails0=df_emails0.reset_index()
text_clean0=[]
for text in df_emails0['body']:
    text_clean0.append(clean_text(text).split())

In [ ]:
# create a list of processed email texts needed to feed into gensim functions
text_clean=[]
for text in df_emails['body']:
    text_clean.append(clean_text(text).split())

# repeat but now with the data frame processed without fowardd email peices and inclusing directory path name and in/out id
text_clean01=[]
df_emails01= df_emails0[0:100]
k=0
for text in text_clean0:
    text_clean01.append([df_emails0['dirpath'][k], df_emails0['inout_id'][k], text])
    k=k+1

In [ ]:
import cPickle
with open('/notebooks/LDA models and data/Data Frames and lists/text_cleanV3.pkl', 'wb') as pickle_file:
    cPickle.dump(obj=text_clean, file=pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    cPickle.dump(obj=text_clean01, file=pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Create dictionary of words (keys) and id's (values) pairs appearing in all emails (the words that we kept)
dictionary = corpora.Dictionary(text_clean)
# Create list of bags of words, one for each cleaned email
text_term_matrix = [dictionary.doc2bow(text) for text in text_clean]

In [ ]:
# we first cretae new nags list by removing empty bags
text_term_matrix_clean=[]
for i in range(0, len(text_term_matrix)):
    if len(np.array(text_term_matrix[i]).shape)==2:
        if np.array(text_term_matrix[i]).shape[1]==2:
            text_term_matrix_clean.append(text_term_matrix[i])  

In [ ]:
#filter low value words
low_value = 0.5

#instantiate empty array to extract our bag of words list text_term_matrix into an array and idem for low tfidf words.
# this will aloow us easily to reduce the decitionary removing only words that are infrequentg in all doecumenmst where
# they appear (only! we need to construct a work around like this as all words appear only in some documents)
low_value_words_list=[]
all_value_words_list=[]
# instantiate newbag of words 
text_term_matrix_tfidf=[]
count_reductions=0
for i in range(0, len(text_term_matrix_clean)):
    bow = text_term_matrix_clean[i]
    low_value_words = []
    # collect low tfidf value words
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    # collect complementary words 
    new_bow = [b for b in bow if b[0] not in low_value_words]
    diff_bow = [b for b in bow if b[0] in low_value_words]
    # update array of low value words and also of all words-this will allow us by aggretation step to determine which words are 
    # low tfidf in all bags of words so dictionary can be reduced for those
    # we need to check in each step if: any words are low, and if anything is left in the bag.
    # We also update new bags list and count how many bags have been reduced, just for checks purpose
    if len(np.array(new_bow).shape)==2:
        if np.array(new_bow).shape[1]==2: 
            # update our new bags of words list
            text_term_matrix_tfidf.append(new_bow)
            
    if len(np.array(diff_bow).shape)==2:
        if np.array(diff_bow).shape[1]==2:
#             low_value_words_array=np.append(arr=low_value_words_array, values=np.array(new_bow), axis=0)
            low_value_words_list.append(diff_bow)
            count_reductions=count_reductions+1
#     all_value_words_array=np.append(arr=all_value_words_array, values=np.array(bow), axis=0)
    all_value_words_list.append(bow)
print count_reductions

In [ ]:
# in this cell we prodcue data frame which contains folowing columns:
# wordid as in our dictionary, number of documents in which this word appears, total number of occurencies and total number of
# emails where the word doc tfidf has value bellow our threshold

# start by transforming lists to appropriate dat frame so we can aggregate
all_value_words_array = np.concatenate([np.array(i) for i in all_value_words_list])
low_value_words_array = np.concatenate([np.array(i) for i in low_value_words_list])

# # remaining lines apply ussual technique of aggregate nad join to selct desired words. We will apply two methods to downsize our 186.000 words dictionary
# so we can do reasonable topic modeling with LDA
all_value_words_df=pd.DataFrame(all_value_words_array, columns=['wordid','count'])

all_value_words_df_count=all_value_words_df.groupby('wordid', as_index=False).count()
all_value_words_df_count.columns=['wordid', 'total_docs']

all_value_words_df_sum=all_value_words_df.groupby('wordid', as_index=False).sum()
all_value_words_df_sum.columns=['wordid', 'total_words']

all_value_words_df=pd.merge(all_value_words_df_sum, all_value_words_df_count, how='inner', on=['wordid'])
###########
low_value_words_df=pd.DataFrame(low_value_words_array, columns=['wordid','count_low'])
low_value_words_df=low_value_words_df.groupby('wordid', as_index=False).agg('count')


all_value_words_df=pd.merge(all_value_words_df, low_value_words_df, how='left', on=['wordid'])

# fix for nan's which appear due to join
import math
def h(x):
    res=0
    if not np.isnan(x):
        res=int(x)
    return res
all_value_words_df['count_low']=all_value_words_df['count_low'].apply(lambda x: h(x))
all_value_words_df.columns=['wordid', 'total_words', 'total_docs', 'total_docs_low_tfidf']

In [ ]:
# In version one of downsizing dictionary (154454 words, which is not feasible for LDA and mpst of these words will be nonsense)
# we choose words based on our tfidf analysis.
df_words_reduced_tfidf = all_value_words_df[all_value_words_df['total_docs']==all_value_words_df['total_docs_low_tfidf']]

# In version two we first take most often arrearing words and then among these we cal again based on number of documents but 
# choosing least often apearing one.
dictionary_size0=all_value_words_df.shape[0] - 10000

df_words_reduced_totwords=all_value_words_df.sort('total_words', ascending=1)[0:dictionary_size0]

# number of words before and after tfidf based reduction; about 15.000 words kept
print all_value_words_df.shape
print df_words_reduced_tfidf.shape

In [ ]:
# define list of words to chunk out by our tfidf method
low_value_words= list(df_words_reduced_tfidf['wordid'])
# update in dictionary
dictionary.filter_tokens(bad_ids=low_value_words)
# compute new bags of wods list of tupels
text_term_matrix = [dictionary.doc2bow(text) for text in text_clean]

In [ ]:
dictionary.save(fname_or_handle='/notebooks/LDA models and data/Data Frames and lists/Dictionary_tfidf_05.pkl')

In [ ]:
text_term_matrix_clean=[]
for i in range(0, len(text_term_matrix)):
    if len(np.array(text_term_matrix[i]).shape)==2:
        if np.array(text_term_matrix[i]).shape[1]==2:
            text_term_matrix_clean.append(text_term_matrix[i])

In [ ]:
import cPickle
with open('/notebooks/LDA models and data/Data Frames and lists/text_term_matrix2.pkl', 'wb') as pickle_file:
    cPickle.dump(obj=text_term_matrix, file=pickle_file, protocol=pickle.HIGHEST_PROTOCOL)